In [1]:
import json
import os
from pathlib import Path
import site
import subprocess
from warnings import warn

In [2]:
from bs4 import BeautifulSoup
import isbnlib
import mysql.connector
import pdfminer
import requests

In [3]:
BASEDIR = Path.home() / 'Development/hw-4.2.0'
site.addsitedir(str(BASEDIR / 'hw'))

In [4]:
from tools import *
from program import Program

In [5]:
os.chdir(BASEDIR)

In [12]:
cnf = configure('etc/config.ini')

In [20]:
(Path.home()/cnf['DATABASE']['pass_file']).read_text()

'pq4nkb3f&zz^m^sql\n'

In [28]:
cnx = mysql.connector.connect(user='fuzzy', password=(Path.home()/cnf['DATABASE']['pass_file']).read_text(),
                              host='192.168.254.71',
                              database='library')
cnx.close()

In [ ]:
public(mysql.connector.connection)

In [ ]:
l = list(map(lambda s: str(s).lstrip('b').strip("'"), [s for s in check_output('pdf2txt.py /home/fuzzy/Library/Python/Zed_Shaw_-_Learn_Python_the_Hard_Way_3rd.pdf'.split()).split(b'\n') if s.startswith(b'ISBN')]))

In [ ]:
isbnlib.meta(l[0].split()[-1])

In [ ]:
args = [ str((Path.home() / 'Library').absolute()) ]

In [ ]:
settings = {'logfile': 'log/pdf.log',
 'program': 'get_info',
 'version': '4.2.0',
 'packages_dir': "'Development/hw-4.2.0'",
 'args_json_file': 'etc/arguments.json',
 'epilog': 'USAGE: None',
 'verbose': True,
 'debug': True,
 'args': args,
 'testing': True,
 'recursive': True}

In [ ]:
class GetPDFInfo(Program):
    def __init__(self, settings):
        super().__init__(settings)
        self.results = list()
        self.process_args()
        
    def process_file(self, p:Path):
        if p.name.endswith('.pdf'):
            l = list()
            try:
                l = list(map(lambda s: str(s).lstrip('b').strip("'"),
                             [s for s in run(['pdf2txt.py', str(p)]).split('\n') if s.startswith('ISBN')]))
            except subprocess.CalledProcessError:
                warn('pdf2txt.py returned an error code!')
            if l:
                info = dict()
                try:
                    info = isbnlib.meta(l[0].split()[-1])
                except isbnlib.NotValidISBNError as e:
                    warn(f'Invalid ISBN in file {p.name} : {l[0].split()[-1]}')
                if info:
                    self.results.append(info)
                else:
                    warn(f'Failed to get info for {p.name}')
            else:
                warn(f'Failed to get ISBN for {p.name}')

In [ ]:
results = GetPDFInfo(settings).results

In [ ]:
results[0]

In [ ]:
RESULTS_FILE = BASEDIR / 'data/info4pybooks.json'
with RESULTS_FILE.open('w') as f:
    json.dump(results, f)

In [ ]:
help(json.dump)

In [ ]:
link = f'http://classify.oclc.org/classify2/ClassifyDemo?search-standnum-txt={results[0]["ISBN-13"]}&startRec=0'

In [ ]:
response = requests.get(link)

In [ ]:
soup = BeautifulSoup(response.text)

In [ ]:
soup.find('tbody').find('td').find('a')#['href']

In [ ]:
OCLC = 'http://classify.oclc.org'

In [ ]:
link = f"{OCLC}{soup.find('tbody').find('td').find('a')['href']}"

In [ ]:
response = requests.get(link)

In [ ]:
soup = BeautifulSoup(response.text)

In [ ]:
ddc = soup.find('tbody').find('tr').find_all('td')[1].text

In [ ]:
DDC_FILE = (BASEDIR / 'data/deweysummaries.txt').read_text().split('\n')

In [ ]:
numbers = [ddc[0] + '00', ddc[:2] + '0', ddc[:3]]

In [ ]:
numbers

In [ ]:
classes = list()
for i, n in enumerate(numbers):
    classes.extend([' '.join(s.split()[1:]) for s in DDC_FILE if s.startswith(numbers[i])])

In [ ]:
classes = [s for s in classes if len(s)]
classes = [numbers[i] + ' ' + classes[i] for i in range(3)]

In [ ]:
classes

In [ ]:
BOOKS_DIR = BASEDIR / 'tests/library/books'

In [ ]:
classdir = BOOKS_DIR / classes[0] / classes [1] / classes[2]

In [ ]:
str(classdir)

In [ ]:
if not classdir.exists():
    os.makedirs(classdir)

In [ ]:
cnx = mysql.connector.connect(user='fuzzy', password='pq4nkb3f&zz^m^sql',
                              host='192.168.254.71',
                              database='library')

In [ ]:
cnx.close()

In [30]:
public(cnx)

autocommit            cmd_stmt_reset           isset_client_flag    
can_consume_results   cmd_stmt_send_long_data  ping                 
charset               collation                python_charset       
close                 commit                   raise_on_warnings    
cmd_change_user       config                   reconnect            
cmd_debug             connect                  reset_session        
cmd_init_db           connection_id            rollback             
cmd_ping              consume_results          server_host          
cmd_process_info      converter                server_port          
cmd_process_kill      cursor                   set_charset_collation
cmd_query             database                 set_client_flags     
cmd_query_iter        disconnect               set_converter_class  
cmd_quit              get_row                  set_login            
cmd_refresh           get_rows                 set_unicode          
cmd_reset_connection  get_server_i

In [31]:
help(cnx.cmd_query)

Help on method cmd_query in module mysql.connector.connection:

cmd_query(query, raw=False, buffered=False, raw_as_string=False) method of mysql.connector.connection.MySQLConnection instance
    Send a query to the MySQL server
    
    This method send the query to the MySQL server and returns the result.
    
    If there was a text result, a tuple will be returned consisting of
    the number of columns and a list containing information about these
    columns.
    
    When the query doesn't return a text result, the OK or EOF packet
    information as dictionary will be returned. In case the result was
    an error, exception errors.Error will be raised.
    
    Returns a tuple()

